# pandas-render

[pandas-render](https://github.com/nok/pandas-render) is a [pandas](https://github.com/pandas-dev/pandas) extension for rendering DataFrames and Series as HTML tables, with support for custom styling and formatting.

Installation:

```bash
pip install pandas-render
```

## Basics

This example shows how to use pandas-render to create a simple HTML table from a pandas DataFrame. It demonstrates how to use Jinja2 templates to format the content of each cell, and how to create links and images in the table.

### Import
Either import pandas with pandas-render explicitly:

In [1]:
from pandas_render import pandas as pd

Or import it implicitly after importing pandas:

```python
import pandas as pd
import pandas_render  # noqa
```

### Data
Create a DataFrame with some sample data. The DataFrame contains information about TV series and movies.

In [2]:
df = pd.DataFrame(
    [
        dict(
            title="Severance",
            id="tt11280740",
            year=2022,
            image_url="https://m.media-amazon.com/images/M/MV5BZDI5YzJhODQtMzQyNy00YWNmLWIxMjUtNDBjNjA5YWRjMzExXkEyXkFqcGc@._V1_QL75_UX380_CR0,4,380,562_.jpg",
            rating=8.7,
            actors=["Adam Scott", "Britt Lower", "Zach Cherry"],
        ),
        dict(
            title="Stranger Things",
            id="tt4574334",
            year=2016,
            image_url="https://m.media-amazon.com/images/M/MV5BMjg2NmM0MTEtYWY2Yy00NmFlLTllNTMtMjVkZjEwMGVlNzdjXkEyXkFqcGc@._V1_QL75_UX380_CR0,0,380,562_.jpg",
            rating=8.6,
            actors=["Millie Bobby Brown", "Finn Wolfhard"],
        ),
        dict(
            title="Python",
            id="tt0209264",
            year=2000,
            image_url="https://m.media-amazon.com/images/M/MV5BZWUwNjExYTYtZmM3ZS00NzA1LWJhNjYtMWExNGEwZTVjNDM3XkEyXkFqcGc@._V1_QL75_UY562_CR8,0,380,562_.jpg",
            rating=3.7,
            actors=["Frayne Rosanoff"],
        ),
    ]
)

Here is the original rendering of the DataFrame. It's a simple table with no formatting or styling. It's not very readable, and the data is not very visually appealing.

In [3]:
df.head()

,title,id,year,image_url,rating,actors
0,Severance,tt11280740,2022,https://m.media-amazon.com/images/M/MV5BZDI5Yz...,8.7,"[Adam Scott, Britt Lower, Zach Cherry]"
1,Stranger Things,tt4574334,2016,https://m.media-amazon.com/images/M/MV5BMjg2Nm...,8.6,"[Millie Bobby Brown, Finn Wolfhard]"
2,Python,tt0209264,2000,https://m.media-amazon.com/images/M/MV5BZWUwNj...,3.7,[Frayne Rosanoff]


### Rendering
Now we can use `render` to create a more visually appealing table. The `render` method takes a dictionary of column names and their corresponding templates. Use the placeholder `{{ content }}` to access the content of each cell. Each value is a [Jinja2](https://github.com/pallets/jinja) template with all features (e.g. filters) to format the content.

In [4]:
df.render(
    templates=dict(
        title="{{ content|upper }}",
        id='<a href="https://www.imdb.com/title/{{ content }}" target="_blank">Link</a>',
        image_url='<img src="{{ content }}" width="100"/>',
        rating="<strong>{{ content }}</strong>",
        actors="<em>{{ content|join(', ') }}</em>",
    ),
)

title,id,year,image_url,rating,actors
SEVERANCE,Link,2022,,8.7,"Adam Scott, Britt Lower, Zach Cherry"
STRANGER THINGS,Link,2016,,8.6,"Millie Bobby Brown, Finn Wolfhard"
PYTHON,Link,2000,,3.7,Frayne Rosanoff


### Filtering and Ordering
You can also filter the columns to be rendered. By default, all columns are rendered. To filter the columns, set `filter_columns=True`. This will only render the columns that are specified in the `columns` dictionary. In addition to filtering, you can also order the columns. The order of the columns in the `columns` dictionary will be used to determine the order of the columns in the rendered table.

In [5]:
df.render(
    templates=dict(
        image_url='<img src="{{ content }}" width="100"/>',
        title="{{ content|upper }}",
    ),
    filter_columns=True,
)

image_url,title
,SEVERANCE
,STRANGER THINGS
,PYTHON


### Dynamic Placeholder
You can also use dynamic placeholders in the templates. It means that you can use each name of a column as a placeholder in the template. This allows you to create templates that are more flexible and can be reused for different columns. For example, you can use `{{ content }}` to access the content of the cell, and `{{ title }}` to access the title of the movie or TV series.
And if you have a column with lists of values, you can use either the filter `join` or a loop to render the content dynamically.

In [6]:
title_template = """
<p><a href="https://www.imdb.com/title/{{ id }}" target="_blank">{{ content|upper }}</a></p>
<p><smaller>Year: {{ year }}</smaller></p>
<p><smaller>Rating: <strong>{{ rating }}</strong></smaller></p>
"""

actors_template = """
{% if content %}
<ol>
    {% for actor in content %}
        <li>{{ actor }}</li>
    {% endfor %}
</ol>
{% else %}
    <p>No actors</p>
{% endif %}
"""

df.render(
    templates=dict(
        image_url='<img src="{{ content }}" width="100"/>',
        title=title_template,
        actors=actors_template,
    ),
    filter_columns=True,
)

image_url,title,actors
,SEVERANCE Year: 2022 Rating: 8.7,Adam Scott Britt Lower Zach Cherry
,STRANGER THINGS Year: 2016 Rating: 8.6,Millie Bobby Brown Finn Wolfhard
,PYTHON Year: 2000 Rating: 3.7,Frayne Rosanoff
